In [14]:
%reload_ext line_profiler
%reload_ext Cython

In [15]:
import sys
import numpy as np
import matplotlib.pyplot as plt
plt.switch_backend("TkAgg")
from scipy.constants import c, e, m_p
from scipy.signal import fftconvolve

from interpolated_functions import interpolated_mod2pi
sin_interpolated = interpolated_mod2pi(np.sin, -2.1 * np.pi, 2.1 * np.pi, 2000000)    
cos_interpolated = interpolated_mod2pi(np.cos, -2.1 * np.pi, 2.1 * np.pi, 2000000)
# import seaborn as sns
# sns.set_context('notebook', font_scale=1.5,
#                 rc={'lines.markeredgewidth': 1})
# sns.set_style('darkgrid', {
#         'axes.linewidth': 2,
#         'legend.fancybox': True})

## VDT sin - sinv - sincos
---

In [46]:
%%cython -I./vdt/include -L./vdt/lib --compile-args=-fopenmp --link-args=-fopenmp --cplus --lib=vdt
#cython boundscheck=False

import os
os.environ["LD_LIBRARY_PATH"] = "./vdt/lib"

cimport cython
import numpy as np
cimport numpy as np
from cython.parallel import prange

cdef extern from "sin.h" namespace "vdt":
    cdef double fast_sin(double x) nogil
    cdef void fast_sinv(int n, double *x, double *s) nogil

cdef extern from "sincos.h" namespace "vdt":
    cdef void fast_sincos(double x, double &s, double &c) nogil

@cython.boundscheck(False)
def vdt_sin(double[::1] x, double[::1] s):
    
    cdef int n = x.shape[0]
    cdef int i
    for i in prange(n, nogil=True, num_threads=1):
        s[i] = fast_sin(x[i])
    
@cython.boundscheck(False)
def vdt_sincos(double[::1] x, double[::1] s, double[::1] c):
    
    cdef int n = x.shape[0]
    cdef int i
    for i in prange(n, nogil=True, num_threads=1):
        fast_sincos(x[i], s[i], c[i])

@cython.boundscheck(False)
def vdt_sinv(double[::1] x, double[::1] s):
    
    cdef int n = x.shape[0]
    fast_sinv(n, &x[0], &s[0])

## math.h sincos
---

In [47]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp

import numpy as np
cimport numpy as np
cimport cython
from cython.parallel import prange

cdef extern from "math.h":
    cdef void sincos(double x, double *sin, double *cos) nogil
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(False)
cpdef fastsincos(double[::1] x, double[::1] s, double[::1] c):
    cdef int n = x.shape[0]
    cdef int i
    for i in prange(n, nogil=True, num_threads=1):
        sincos(x[i], &s[i], &c[i])

## cmath sin
---

In [48]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp

import numpy as np
cimport numpy as np
cimport cython
from cython.parallel import prange
from libc.math cimport sin

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(False)
def cmsin(double[::1] x, double[::1] s):
    
    cdef int n = x.shape[0]
    cdef int i
    for i in prange(n, nogil=True, num_threads=1):
        s[i] = sin(x[i])

In [49]:
n = int(1e6)
x = np.linspace(0, 2*np.pi, n)
xd = np.zeros(len(x)*2)
a = np.zeros(n)
b = np.zeros(n)

d = np.zeros(2*n)
xd[::2] = x[:]

# a.data[:]

In [50]:
%timeit np.cos(x)
%timeit sin_interpolated(x)
# %timeit fastsincos(xd, d)
%timeit cmsin(x, a)
%timeit vdt_sin(x, a)
%timeit vdt_sinv(x, a)
%timeit vdt_sincos(x, a, b)

10 loops, best of 3: 29 ms per loop
100 loops, best of 3: 18.8 ms per loop
10 loops, best of 3: 27.3 ms per loop
100 loops, best of 3: 11.4 ms per loop
100 loops, best of 3: 7.96 ms per loop
100 loops, best of 3: 17.9 ms per loop


In [24]:
a1 = np.sin(x)
cmsin(x, a); a2 = a.copy()
vdt_sinv(x, a); a3 = a.copy()
a4 = sin_interpolated(x)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
ax1.plot(x, a1-a2)
ax2.plot(x, a1-a3)
ax3.plot(x, a1-a4)
# plt.plot(x, b)
plt.show()

***

In [36]:
%%cython --lib=gsl --lib=gslcblas

import numpy as np
cimport numpy as np
cimport cython

cdef extern from "gsl/gsl_sf_trig.h":
    cdef double gsl_sf_sin(double x)
    
def gsl_sin(double[::1] x, double[::1] s):
    
    cdef int n = x.shape[0]
    cdef int i
    for i in xrange(n):
        s[i] = gsl_sf_sin(x[i])

#     sc = sincos(x, &s[0], &c[0])

# cpdef sc(double x, np.ndarray[double, ndim=1] s, np.ndarray[double, ndim=1] c):
#     return sincos(x, &s[0], &c[0])
#     return sincos(x, <double*> s.data, <double*> c.data)

In [12]:
from PyHEADTAIL.particles.slicing import UniformBinSlicer
from PyHEADTAIL.impedances.wakes import Resonator, WakeField
from SPS import SPS


macroparticlenumber = int(10e4)
intensity = 2e11
epsn_x = 2e-6
epsn_y = 2e-6
sigma_z = 0.23


sps = SPS(machine_configuration='Q20-injection', n_segments=1, longitudinal_mode='linear',
          Qp_x=5, Qp_y=5)

bunch = sps.generate_6D_Gaussian_bunch(
    macroparticlenumber, intensity, epsn_x, epsn_y, sigma_z)

sz = bunch.sigma_z()
slices_for_wakes = UniformBinSlicer(100, z_cuts=(-3*sz, 3*sz))
reso = Resonator(7.5e6, 1.3e9, 1, 1, 0, 0, 0, switch_Z=False)
wake = WakeField(slices_for_wakes, reso)
sps.one_turn_map.append(wake)

m1, m2, m3 = sps.one_turn_map

Synchrotron init. From kwargs: n_segments = 1
Synchrotron init. From kwargs: Qp_y = 5
Synchrotron init. From kwargs: Qp_x = 5
Synchrotron init. From kwargs: longitudinal_mode = 'linear'
*** PyHEADTAIL WARNING! Acceleration not handled properly by this kind of convolution due to changing bunch length!


In [13]:
%timeit m1.track(bunch)
%timeit m2.track(bunch)
%timeit m3.track(bunch)

10 loops, best of 3: 18.1 ms per loop
1000 loops, best of 3: 1.82 ms per loop
100 loops, best of 3: 3.6 ms per loop


In [ ]:
# plt.ion()
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(21,9))
mx = []
ex = []
for i in xrange(10000):
    for m in sps.one_turn_map:
        m.track(bunch)

    mx.append(bunch.mean_x())
    ex.append(bunch.epsn_x())
    if not i%500==0: continue
    print "\rTurn # {:d}".format(i),
    print

    # ax1.plot(bunch.x, bunch.xp, '.')
    # ax2.plot(bunch.y, bunch.yp, '.')
    # ax3.plot(bunch.z, bunch.dp, '.')
    # ax1.set_xlim(-1e-2, 1e-2)
    # ax2.set_xlim(-1e-2, 1e-2)
    # ax3.set_xlim(-1, 1)
    # ax1.set_ylim(-2e-4, 2e-4)
    # ax2.set_ylim(-2e-4, 2e-4)
    # ax3.set_ylim(-1e-2, 1e-2)

    # plt.draw()
    # plt.pause(0.1)
    # [ax.cla() for ax in [ax1, ax2, ax3]]

print
ax1.plot(mx)
ax2.plot(ex)
plt.show()
wurstel


# Here we can play with **tunes** and **n_segments** - really the combination of the two, plus the type of non-linearity, determine the resonances excited...

# In[7]:

# =====================================
# Input parameters
Qx, Qy = 20.24, 20.18
n_segments = 3#9
n_turns = 2048

# =====================================
# Build machine
C = np.pi*50*44
# sps = SPS(machine_configuration='Q20-flattop', RF_at='end_of_transverse', optics_mode='non-smooth',
#           name=['s{:d}'.format(i) for i in xrange(n_segments+1)],
#           s=[i*C/n_segments for i in xrange(n_segments+1)],
#           accQ_x=[i*Qx/n_segments for i in xrange(n_segments+1)],
#           accQ_y=[i*Qy/n_segments for i in xrange(n_segments+1)],
#           beta_x=np.array([50, 100, 10, 50]), beta_y=np.array([50, 10, 100, 50]),
#           alpha_x=np.zeros(n_segments+1), alpha_y=np.zeros(n_segments+1)
#           )

sps.one_turn_map.pop(-1)
ax = sps.transverse_map.alpha_x[0]
ay = sps.transverse_map.alpha_y[0]
bx = sps.transverse_map.beta_x[0]
by = sps.transverse_map.beta_y[0]

# =====================================
# Add octupoles
from PyHEADTAIL.multipoles.multipoles import ThinSextupole, ThinOctupole

# sex = ThinSextupole(k2l=2)
# sps.install_after_each_transverse_segment(sex)
oct = ThinOctupole(k3l=1000)
sps.install_after_each_transverse_segment(oct)


# In[8]:

# bunch.update({
#         'x': xx.flatten(),
#         'xp': xxp.flatten(),
#         'y': yy.flatten(),
#         'yp': yyp.flatten()})

x = np.zeros((n_turns, macroparticlenumber))
xp = np.zeros((n_turns, macroparticlenumber))
y = np.zeros((n_turns, macroparticlenumber))
yp = np.zeros((n_turns, macroparticlenumber))

# ===============
# Do the tracking
for i in xrange(n_turns):
    for m in sps.one_turn_map:
        m.track(bunch)

    x[i,:] = bunch.x
    xp[i,:] = bunch.xp
    y[i,:] = bunch.y
    yp[i,:] = bunch.yp
    
# =======================
# Some frequency analysis
from PySussix import Sussix

tunesx1 = np.zeros(macroparticlenumber)
tunesy1 = np.zeros(macroparticlenumber)
tunesx2 = np.zeros(macroparticlenumber)
tunesy2 = np.zeros(macroparticlenumber)

ssx = Sussix()

ssx.sussix_inp(nt1=1, nt2=n_turns/2, idam=2, ir=0, tunex=0.2, tuney=0.2)
for i in xrange(macroparticlenumber):
    if i%20==0:
        sys.stdout.write("\r--> Processing particle {:d}".format(i))
    ssx.sussix(x[:,i], bx*xp[:,i], y[:,i], by*yp[:,i], x[:,i], xp[:,i])
    tunesx1[i] = ssx.ox[np.argmax(ssx.ax)]
    tunesy1[i] = ssx.oy[np.argmax(ssx.ay)]

ssx.sussix_inp(nt1=n_turns/2, nt2=n_turns, idam=2, ir=0, tunex=0.2, tuney=0.2)
for i in xrange(macroparticlenumber):
    if i%20==0:
        sys.stdout.write("\r--> Processing particle {:d}".format(i))
    ssx.sussix(x[:,i], bx*xp[:,i], y[:,i], by*yp[:,i], x[:,i], xp[:,i])
    tunesx2[i] = ssx.ox[np.argmax(ssx.ax)]
    tunesy2[i] = ssx.oy[np.argmax(ssx.ay)]
    
diff_x = np.log(tunesx2-tunesx1)
diff_y = np.log(tunesy2-tunesy1)


plt.close('all')
# ===============
# Plot output
fig = plt.figure(figsize=(20, 10))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132, sharex=ax1, sharey=ax1)
ax3 = fig.add_subplot(133)

ax1.plot(x, xp, '.')
ax2.plot(y, yp, '.')
ax1.set_xlim(-1e-2, 1e-2)
ax1.set_ylim(-1e-4, 1e-4)

ax3.scatter(tunesx1, tunesy1, c=diff_x, s=40, marker='o', cmap='viridis')
ax3.set_xlim(left=Qx%1-4e-2)
ax3.set_ylim(bottom=Qy%1-4e-2)

plt.show()


# ***

# ## Regular points

# In[119]:

bunch = sps.generate_6D_Gaussian_bunch(n_macroparticles=1e5, intensity=1e11,
                                       epsn_x=4.5e-6, epsn_y=4.5e-6, sigma_z=0.23)
sJx = np.sqrt(bunch.sigma_x()**2 + (bx*bunch.sigma_xp())**2)/2.
sJy = np.sqrt(bunch.sigma_x()**2 + (bx*bunch.sigma_xp())**2)/2.
print sJx, sJy


J = np.linspace(0, 1, 12)**2 * 3.6e-8
phi1 = np.linspace(0, 2*np.pi, 12)
phi2 = np.linspace(0, 2*np.pi, 12)
phi3 = np.linspace(0, 2*np.pi, 12)

JJ, PP = np.meshgrid(J, phi1)
xx = np.sqrt(2.*JJ*bx)*np.cos(PP)
xxp = -np.sqrt(2.*JJ/bx)*(np.sin(PP) + ax*np.cos(PP))
yy = np.sqrt(2.*JJ*by)*np.sin(PP)
yyp = -np.sqrt(2.*JJ/by)*(np.cos(PP) + ay*np.sin(PP))


macroparticlenumber = np.product(JJ.shape)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,9))
ax1.plot(xx, yy, '-o')
ax2.plot(yyp, xxp, '-o')

plt.show()


# ## Build a 4-sphere grid

# In[123]:

macroparticlenumber = 400

u4 = np.random.normal(size=(4, macroparticlenumber))
r  = np.sqrt(sum(u4**2))
u4 *= 1./r * 7e-5
xx = u4[0,:]
xxp = u4[1,:]
yy = u4[2,:]
yyp = u4[3,:]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,9))
ax1.plot(xx, yy, 'o')
ax2.plot(xxp, yyp, 'o')

plt.show()


# In[100]:

bunch = sps.generate_6D_Gaussian_bunch(n_macroparticles=1e5, intensity=1e11,
                                       epsn_x=4.5e-6, epsn_y=4.5e-6, sigma_z=0.23)
sJx = np.sqrt(bunch.sigma_x()**2 + (bx*bunch.sigma_xp())**2)/2.
sJy = np.sqrt(bunch.sigma_x()**2 + (bx*bunch.sigma_xp())**2)/2.
print sJx, sJy


J = np.linspace(0, 1, 12)**2 * 3.6e-8
phi1 = np.linspace(0, 2*np.pi, 12)
phi2 = np.linspace(0, 2*np.pi, 12)
phi3 = np.linspace(0, 2*np.pi, 12)

JJ, PP = np.meshgrid(J, phi1)
xx = np.sqrt(2.*JJ*bx)*np.cos(PP)
xxp = -np.sqrt(2.*JJ/bx)*(np.sin(PP) + ax*np.cos(PP))
yy = np.sqrt(2.*JJ*by)*np.cos(PP)
yyp = -np.sqrt(2.*JJ/by)*(np.sin(PP) + ay*np.cos(PP))


macroparticlenumber = np.product(JJ.shape)

x = np.zeros((n_turns, macroparticlenumber))
xp = np.zeros((n_turns, macroparticlenumber))
y = np.zeros((n_turns, macroparticlenumber))
yp = np.zeros((n_turns, macroparticlenumber))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,9))
ax1.plot(xx, yy, '-o')
ax2.plot(yy, yyp, '-o')

plt.show()


# ***